# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [1]:
%%time
import pandas as pd
import numpy as np
import datetime
import clickhouse_connect

CPU times: total: 516 ms
Wall time: 555 ms


---

#  <span style="color:Blue">Juntar Ficheiros</span>

- Importar do Clickhouse

In [2]:
client = clickhouse_connect.get_client(host='e28fluocjc.europe-west4.gcp.clickhouse.cloud', 
                                       port=8443, 
                                       username='default', 
                                       password='eKn4CWkTDFpi_',
                                       database='Delta')

In [3]:
#Ler o ficheiro

dfClick = client.query_df('SELECT * FROM FinalDataBaseDelta2023')

dfClick = dfClick[dfClick.DATA != dfClick.DATA.unique()[0]]

- Importar ficheiros novos

In [ ]:
dfNovo = pd.read_excel("D:\\B&N Dados\\Delta\\Diário\\dia_actual.xlsb") #xlsx


dfNovo = dfNovo[['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                         "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)"]]

# se xlsx
#dfNovo['DATA'] = pd.to_datetime(dfNovo['DATA'], format='%d-%m-%Y') 

# se xlsb
dfNovo['DATA'] = pd.to_datetime(dfNovo['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

- Produtos específicos

In [3]:
# Passar produtos e lojas para listas

produtos = dfClick.DESC_ARTIGO.unique().tolist()
lojas = dfClick.STORE.unique().tolist()

# Alterar o dataframe para apenas incluir os produtos e lojas em causa

dfNovo = dfNovo[(dfNovo["DESC_ARTIGO"].isin(produtos)) & (dfNovo["STORE"].isin(lojas))].copy()

# Renomear colunas e criar as do dia

dfNovo = dfNovo.rename(columns={"VND (D-1)": "SELLOUT_1_Dias_Antes", "SOH": "STOCK_1_Dias_Antes"})
dfNovo['SELLOUT'] = dfNovo.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)
dfNovo['STOCK'] = dfNovo.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)

CPU times: total: 0 ns
Wall time: 433 ms


- Juntar

In [ ]:
dfDiario = pd.concat(["dfClick", "dfNovo"], ignore_index=True)

# <font color=green>Ficheiro Lido<font>

---

# Colunas de métricas interessantes

> - ROTURA

In [7]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)

> - PRÉ_ROTURA

In [8]:
# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)

# Colunas de métricas 30, 60, 120 e 180 dias antes

In [11]:
diasMetHist = [30, 60, 120, 180]

> - Volatilidade de Procura: <br>
coeficiente de variação

In [12]:
dfFinal=dfClick

In [13]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Volatilidade_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [14]:
%%time
dfFinal["Volatilidade_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))

dfFinal["Volatilidade_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))

dfFinal["Volatilidade_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).std())/
                              dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))

dfFinal["Volatilidade_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).std())/
                              dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))



CPU times: total: 62.5 ms
Wall time: 59.9 ms


> - Percentagem de Rotura: <br>
média de roturas $* 100$

In [ ]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100

In [10]:
%%time
#historico("Percentagem_Roturas", "ROTURA", pd.Series.mean)

dfFinal["Percentagem_Roturas_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


CPU times: total: 2.19 s
Wall time: 2.12 s


> - Percentagem de Supply:<br>
média de vezes que foi pedido stock $*100$

In [11]:
dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED"].shift(1)==0) & (dfFinal["EXPECTED"]>0), 1, 0)

In [ ]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Supply_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [12]:
#historico("Percentagem_Supply", "New_Supply", pd.Series.mean)

dfFinal["Percentagem_Supply_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum())/30)*100

dfFinal["Percentagem_Supply_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum())/60)*100

dfFinal["Percentagem_Supply_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum())/120)*100

dfFinal["Percentagem_Supply_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum())/180)*100


> - Efeito fim de semana

In [13]:
dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()

In [ ]:
for i in diasMetHist:
    dfFinal[f"Efeito_Fds_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).median())))-1


In [14]:
dfFinal["Efeito_Fds_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())))-1

dfFinal["Efeito_Fds_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())))-1

dfFinal["Efeito_Fds_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())))-1

dfFinal["Efeito_Fds_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())))-1

> - Tempo médio inter-supply

In [15]:
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

In [ ]:
for i in diasMetHist:
    dfFinal[f"InterSupplyMed_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [16]:
#historico("InterSupplyMed", "InterSupply", pd.Series.mean)

dfFinal["InterSupplyMed_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Tempo indisponível

In [17]:
'''O que vai acontecer é: no primeiro dia em que há rotura vai aparecer a soma de todos os dias com rotura a seguir a esse!
Todos os outros valores serão NaN para não serem considerados quando for feita a média. Assim a média corresponderá ao
número médio de dias em que se deixa um produto em rotura.'''

dfFinal["Tempo_Indisponível"] = np.where(dfFinal["ROTURA"]==1, 1, 0)


In [ ]:
for i in diasMetHist:
    dfFinal[f"Tempo_Indisponível_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [18]:
#historico("Percentagem_Supply", "New_Supply", pd.Series.mean)

dfFinal["Tempo_Indisponível_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Tempo_Indisponível_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Tempo_Indisponível_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Tempo_Indisponível_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de dias em Stock Borderline

In [19]:
dfFinal["Stock_Borderline"] = np.where(dfFinal["STOCK"]<0.2*dfFinal["PRES_STOCK"], 1, 0)

In [ ]:
for i in diasMetHist:
    dfFinal[f"Tempo_Stock_Borderline_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [20]:
dfFinal["Tempo_Stock_Borderline_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de dias de Linear Incompleto

In [21]:
dfFinal["Linear_Incompleto"] = np.where(dfFinal["STOCK"]<dfFinal["PRES_STOCK"], 1, 0)

In [ ]:
for i in diasMetHist:
    dfFinal[f"Tempo_Linear_Incompleto_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [22]:
dfFinal["Tempo_Linear_Incompleto_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de dias sem vendas

In [23]:
dfFinal["Sem_Vendas"] = np.where(dfFinal["SELLOUT"] == 0, 1, 0)

In [ ]:
for i in diasMetHist:
    dfFinal[f"Sem_Vendas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [24]:
#historico("Sem_Vendas", "Sem_Vendas", pd.Series.mean)

dfFinal["Sem_Vendas_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Vendas perdidas

In [25]:
dfFinal['ROTURA_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["ROTURA"].copy()
dfFinal['ROTURA_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["ROTURA"].copy()

In [ ]:
for i in diasMetHist:
    dfFinal[f"Vendas_Perdidas_{i}"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).median())) + 
                                       (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=i, min_periods=1).median())))
    

In [26]:
'''Mediana de fins de semana e mediana de semana a multiplicar pelo nº de dias em que há rotura, soma dos valores para
ter as perdas de vendas estimadas'''

#mediana fds*roturas fds + mediana semana*roturas semana

dfFinal["Vendas_Perdidas_30"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())) + 
                                (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_60"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())) + 
                                (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_120"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())) + 
                                 (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_180"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())) + 
                                 (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median()))
)

# Métricas até 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [28]:
# Quantos dias antes:

diaI=4         #dia inicial
diaF=5       #dia final

diasMet = [4, 5, 10]

> Função

In [29]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [30]:
%%time
# Usar função para sellouts até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 1.66 s
Wall time: 1.66 s


> - STOCKS

In [31]:
# Usar função para Stocks até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "STOCK")

> > - Ordenar

> - INTRANSIT e EXPECTED

In [32]:
# Usar função para Trânsito até 10 dias antes


for i in diasMet:
    dias(dfFinal, i, "INTRANSIT")
    
for i in diasMet:
    dias(dfFinal, i, "EXPECTED")

> - STK

In [33]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in diasMet:
    dias(dfFinal, i, "STK")

> - MSA

In [ ]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10, min_periods=1).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10, min_periods=1).std())

for i in diasMet:
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20, min_periods=1).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20, min_periods=1).std())
  
for i in diasMet:
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [35]:
# Coluna de Ciclos de reposição

dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]

for i in diasMet:
    dias(dfFinal, i, "CICLOS")

> - Dias para rotura de Stock

In [36]:
#dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STOCK"] / dfFinal["MSA10"]

for i in diasMet:
    dias(dfFinal, i, "Dias_para_Rotura_Stock")

> - Dias para rotura de Linear

In [37]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duração_Linear'] = dfFinal["PRES_STOCK"] / dfFinal["MSA10"]

for i in diasMet:
    dias(dfFinal, i, "Dias_Duração_Linear")

> - Adequação de Stock

In [38]:
# Coluna de adequação de stock


dfFinal["Adequação"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in diasMet:
    dias(dfFinal, i, "Adequação")

> - Smart é com semana e fim de semana

In [ ]:
%%time
import pandas as pd
from datetime import datetime, timedelta


# Define a function to calculate weekday and weekend counts
def calculate_weekday_weekend_counts(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(5):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA": weekday_count, "CONTAGEM_FIMSEMANA": weekend_count})

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA", "CONTAGEM_FIMSEMANA"]] = dfFinal["DATA"].apply(calculate_weekday_weekend_counts)



dfFinal["Balance_Smart_5_Dias_Antes"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].transform(lambda x: x.rolling(window=30, min_periods=1).median())
     * dfFinal['CONTAGEM_FIMSEMANA'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].transform(lambda x: x.rolling(window=30, min_periods=1).median())
     * dfFinal['CONTAGEM_SEMANA']))
     / dfFinal["STK"])

# <font color=blue>Escrever

- Dias certos

In [73]:
# Ficheiro Dia
dfEscreverDia = dfFinal[dfFinal.DATA == dfFinal.DATA.unique().tolist()[-1]].copy()

#Ficheiro Mês
dfEscreverMes = dfFinal[dfFinal.DATA >= dfClick.DATA.unique().tolist()[-30]].copy()

- Passar para csv

# Clickhouse

In [28]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

In [45]:
#Estabelecer a base a ser lida

baseDay = dfEscreverDia
baseMonth = dfEscreverMes

tabelaDay = "Day"
tabelaMonth = "Month"

In [56]:

client = clickhouse_connect.get_client(host='e28fluocjc.europe-west4.gcp.clickhouse.cloud', 
                                       port=8443, 
                                       username='default', 
                                       password='N_Mx30OFTC1hN',
                                       database='Delta')



###
baseDay['DATA']= pd.to_datetime(baseDay['DATA'], format='%Y-%m-%d')  # Passar para datetime
new_columns = [unidecode(col) for col in baseDay.columns]           # Tirar acentos e afins
baseDay.columns = new_columns                                       # Aplicar alterações da linha anterior
###

# Tipos de dados
data = ["DATA"]
texto = [col for col in baseDay.columns if baseDay[col].dtype == 'object']
inteiros = [col for col in baseDay.columns if baseDay[col].dtype in ['int64', 'int32']]
floats = [col for col in baseDay.columns if baseDay[col].dtype == 'float64']


# Só floats é que permitem missing values
for col_name in inteiros:
    baseDay[col_name] = baseDay[col_name].astype(float)
# Missing values em strings estragam tudo
baseDay[texto] = baseDay[texto].fillna("-")


def schema(lista, tipo):
    result_list = [f"{element} {tipo}" for element in lista]
    return result_list

data1 = schema(data, "Date")
texto1 = schema(texto, "String")
inteiros1 = schema(inteiros, "Float64")
floats1 = schema(floats, "Float64")
total = tuple(data1 + texto1 + inteiros1 + floats1)

schema = ', '.join([column.replace("'", "") for column in total])

# Split the input string by commas
parts = schema.split(', ')
# Process each part and wrap the first word in double quotes
output_parts = []
for part in parts:
    words = part.split()
    if words:
        first_word = words[0]
        remaining_words = ' '.join(words[1:])
        output_part = f'"{first_word}" {remaining_words}'
        output_parts.append(output_part)
# Join the modified parts back into a string
schema = ', '.join(output_parts)



# Eliminar tabela no CH
client.command(f'DROP TABLE IF EXISTS {tabela}')

# Criar tabela no CH
client.command(f'''
    CREATE TABLE IF NOT EXISTS {tabela} (
        {schema}
        ) ENGINE = MergeTree
        ORDER BY (DATA)
''')

client.insert_df(tabela, baseDay, column_names=baseDay.columns.tolist())

In [61]:
dfFinal.head()

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,...,Balance_Optimized_5_Dias_Antes,Balance_Mediano,Balance_Mediano_4_Dias_Antes,Balance_Mediano_5_Dias_Antes,Balance_Liberal,Balance_Liberal_4_Dias_Antes,Balance_Liberal_5_Dias_Antes,Balance_Conservador,Balance_Conservador_4_Dias_Antes,Balance_Conservador_5_Dias_Antes
0,2022-01-01,5.609060e+12,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1.0,CNT MATOSINHOS,0.0,48.0,120.0,151.0,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,5.609060e+12,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1.0,CNT MATOSINHOS,0.0,48.0,120.0,137.0,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,5.609060e+12,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1.0,CNT MATOSINHOS,48.0,0.0,120.0,175.0,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,5.609060e+12,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1.0,CNT MATOSINHOS,0.0,0.0,120.0,162.0,175.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-05,5.609060e+12,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1.0,CNT MATOSINHOS,0.0,48.0,120.0,195.0,162.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
